# Additional Tests for the Simulation

## Imports

In [13]:
from simulation.classes import Airport, Queue, Passenger
from simulation.helpers import run_simulation
import numpy as np

## Test 1
If the passengers are being allocated to the additional service 3% of the time

In [14]:
queue = Queue(timestamp=0, service_rate=1/2)
additional_service_count = 0
run_until = 10000 # number of samples

for i in range(run_until):
    # create a new passenger for each trial
    passenger = Passenger(id=i, arrival_time=0)
    if passenger.needs_additional_screening:
        additional_service_count += 1
        
proportion = additional_service_count / run_until
print(f"Proportion needing additional screening: {proportion:.2f}")

# Calculate confidence interval
z_score = 1.96  # for 95% confidence
margin_of_error = z_score * np.sqrt(proportion * (1 - proportion) / run_until)
confidence_interval = (proportion - margin_of_error, proportion + margin_of_error)
print(f"95% confidence interval: {confidence_interval}")

Proportion needing additional screening: 0.03
95% confidence interval: (0.026278170628343474, 0.03292182937165653)


In [15]:
# Alternative way to test this
avg_queue_lens, arrival_times, all_passengers, max_queue_length = run_simulation(arrival_rate=10, service_rate=1/2, service_distribution=None, additional_screening_distribution=None, run_until=1000, num_queues=3)

# Count passengers who needed additional screening
additional_screening_count = sum(1 for p in all_passengers.values() if p.needs_additional_screening)

total_passengers = len(all_passengers)
proportion = additional_screening_count / total_passengers if total_passengers > 0 else 0

print(f"Passengers needing additional screening: {additional_screening_count}/{total_passengers} ({proportion:.1%})")

Passengers needing additional screening: 31/1000 (3.1%)


## Test 2
Check whether passengers are being assigned to the shortest queue

In [16]:
airport = Airport(arrival_rate=10, service_rate=1/2, num_queues=3)

# creating passenger objects
passenger1 = Passenger(1, arrival_time=0)
passenger2 = Passenger(2, arrival_time=1)
passenger3 = Passenger(3, arrival_time=2)
passenger4 = Passenger(4, arrival_time=3)
passenger5 = Passenger(5, arrival_time=4)
passenger6 = Passenger(6, arrival_time=5)

# Add passengers to queues manually to test shortest queue selection
airport.passenger_queues[0].waiting = [passenger1, passenger2]  # 2 passengers
airport.passenger_queues[1].waiting = [passenger3]  # 1 passenger
airport.passenger_queues[2].waiting = [passenger4, passenger5, passenger6]  # 3 passengers

# Find shortest queue
shortest_queue = airport.find_shortest_queue()
shortest_index = airport.passenger_queues.index(shortest_queue)
print(f"Shortest queue index: {shortest_index}, with {len(shortest_queue.waiting)} passengers waiting")

# Verify it's actually the shortest
queue_lengths = [q.total_passengers() for q in airport.passenger_queues]
print(f"All queue lengths: {queue_lengths}")
print(f"Test passed: {queue_lengths[shortest_index] == min(queue_lengths)}")

Shortest queue index: 1, with 1 passengers waiting
All queue lengths: [2, 1, 3]
Test passed: True


## Test 3
Check if passengers are randomly assigned to a queue when all queues are the same length. Here, we want to test if the proportion of passengers assigned to each queue is roughly equal over many trials. 

In [17]:
airport = Airport(arrival_rate=10, service_rate=1/2, num_queues=3)

# creating passenger objects
passenger1 = Passenger(1, arrival_time=0)
passenger2 = Passenger(2, arrival_time=1)
passenger3 = Passenger(3, arrival_time=2)
passenger4 = Passenger(4, arrival_time=3)
passenger5 = Passenger(5, arrival_time=4)
passenger6 = Passenger(6, arrival_time=5)

# Add passengers to queues manually to test shortest queue selection
airport.passenger_queues[0].waiting = [passenger1, passenger2]  # 2 passengers
airport.passenger_queues[1].waiting = [passenger3, passenger4]  # 2 passengers
airport.passenger_queues[2].waiting = [passenger5, passenger6]  # 2 passengers

num_trials = 1000
selected_indices = []

for _ in range(num_trials):
    selected_queue = airport.find_shortest_queue()
    selected_index = airport.passenger_queues.index(selected_queue)
    selected_indices.append(selected_index)

# Count how many times each queue was selected
queue_0_count = selected_indices.count(0)
queue_1_count = selected_indices.count(1)
queue_2_count = selected_indices.count(2)

perc_1 = (queue_0_count/num_trials)*100
perc_2 = (queue_1_count/num_trials)*100
perc_3 = (queue_2_count/num_trials)*100

print(f"Queue selection distribution over {num_trials} trials:")
print(f"Queue 0: {queue_0_count} times ({perc_1:.1f}%)")
print(f"Queue 1: {queue_1_count} times ({perc_2:.1f}%)")
print(f"Queue 2: {queue_2_count} times ({perc_3:.1f}%)")

Queue selection distribution over 1000 trials:
Queue 0: 333 times (33.3%)
Queue 1: 322 times (32.2%)
Queue 2: 345 times (34.5%)


## Test 4
Testing queue placing dynamics, meant to check whether:
* There are unique arrival and departure events for each new passenger
* When a passenger arrives, they are placed on the shortest queue or assigned at random when all queues have the same length
* When a passenger leaves, according to the scheduled time, they leave the queue completely

In [18]:
# Parameters
run_until = 100
num_queues = 3
avg_waiting_times = []
arrival_rate = 10 # minutes
service_rate = 1/2 # minutes

airport = Airport(arrival_rate, service_rate, num_queues)

# Schedule the specified number of arrivals
current_time = 0
for _ in range(run_until):
    airport.add_arrival(current_time)
    current_time = airport.curr_arrival_time  # Use the scheduled arrival time for next scheduling

# Initial schedule before any service
airport.print_schedule() 
    
while airport.priority_q:
    airport.run_next_service()
    airport.print_schedule()

Airport at time (minutes) 0: 0 total waiting, 0 total being served
  Queue 1: 0 waiting, 0 being served.
  Queue 2: 0 waiting, 0 being served.
  Queue 3: 0 waiting, 0 being served.
  Additional Screening: 3 needed it so far.
	[Arrival]    t=0.21: Passenger 1 → Queue None
	[Arrival]    t=0.28: Passenger 2 → Queue None
	[Arrival]    t=0.31: Passenger 3 → Queue None
	[Arrival]    t=0.31: Passenger 4 → Queue None
	[Arrival]    t=0.37: Passenger 5 → Queue None
	[Arrival]    t=0.60: Passenger 6 → Queue None
	[Arrival]    t=0.70: Passenger 7 → Queue None
	[Arrival]    t=0.72: Passenger 8 → Queue None
	[Arrival]    t=0.77: Passenger 9 → Queue None
	[Arrival]    t=0.85: Passenger 10 → Queue None
	[Arrival]    t=0.95: Passenger 11 → Queue None
	[Arrival]    t=0.99: Passenger 12 → Queue None
	[Arrival]    t=1.04: Passenger 13 → Queue None
	[Arrival]    t=1.07: Passenger 14 → Queue None
	[Arrival]    t=1.13: Passenger 15 → Queue None
	[Arrival]    t=1.25: Passenger 16 → Queue None
	[Arrival]    t=